## Sistemas de Inteligencia Artificial
### TP0: Introducción al Análisis de Datos
###### Primer Cuatrimestre 2023
#### Introducción

El objetivo de este TP es evaluar una función que depende de varios parámetros de entrada, fundamentando las conclusiones con gráficos pertinentes y explicando la metodología utilizada para llegar a cada una. Para ello será provisto un código fuente que incluye una implementación de dicha función junto con ejemplos de ejecución.
#### Función a Evaluar
La función a evaluar es la de captura de un Pokemon, consistente en dos parámetros de entrada: el Pokemon que se está intentando capturar y el tipo de pokebola que se pretende utilizar. La misma retorna un valor booleano correspondiente al resultado del intento de captura.
El éxito de la captura depende del estado en el que se encuentre el Pokemon y del tipo de pokebola que se utilice. Además, algunos Pokemones son intrínsecamente más fáciles de capturar que otros. Para evaluar la función correctamente se deberan probar varias configuraciones de entrada y analizar los resultados correspondientes.
#### Contexto
Pokemon es una franquicia de videojuegos donde el jugador se dedica a armar un pequeño equipo de monstruos (llamados pokemones), entrenarlos peleando contra otros pokemones con el objetivo de convertirse en el mejor entrenador de pokemones de la región. Para conseguir estos pokemones el jugador tiene que explorar el medio salvaje donde estos habitan, pelear contra ellos y finalmente capturarlos utilizando herramientas particulares llamadas pokebolas.
#### Archivos Provistos
Toda la implementación provista se encuentra hecha en Python y contiene lo necesario para poder ejecutar la función y evaluar sus resultados.
### pokemon.py
Modelo de un Pokemon y la implementación de un PokemonFactory para generar nuevos Pokemones. Cada Pokemon cuenta con propiedades que son constantes e inmutables para todos los miembros de la especie y otras que son mutables y corresponden al estado actual de la instancia.
###### Propiedad Tipo Descripción
* current hp int Vida actual (Min: 0)
* level int Nivel de experiencia (Min: 1, Max: 100)
* status effect Enum(StatusEffect) Condici´on de salud
### pokeball.py
Modelo genérico de una pokebola y la implementación concreta de 4 pokebolas distintas:
“PokeBall”, “UltraBall”, “FastBall” y “HeavyBall”.
### pokemon.json
Archivo con los parámetros inmutables de 5 especies distintas de Pokemones. Cada Pokemon cuenta con las siguientes propiedades inmutables:
###### Propiedad Tipo Descripción
* type string[2] Especie de Pokemon (Ej: [ROCK”, ”FIRE”] o [”FIRE, ”NONE”] )
* stats int[6] [base hp, attack, defense, Sp. attack, Sp. defense, speed]
* wieght float Peso del Pokemon
* catch rate int Dificultad de captura. (Min: 0, Max: 255)

### catching.py
Implementación de la función attempt_catch que es la *función a evaluar*.
* Input: Pokemon a capturar y el tipo de pokebola a utilizar
* Output: un valor de tipo boolean (attempt success) que indica si el Pokemon fue capturado o no y otro valor de tipo float (capture rate) que indica la probabilidad de que el pokemon sea capturado, dependiendo de un valor aleatorio.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

from src.catching import attempt_catch
from src.pokemon import PokemonFactory, StatusEffect


### Preguntas Guía
Se desea evaluar qué factores influyen en la captura de un Pokemon teniendo en cuenta los parámetros inmutables del mismo junto con la pokebola utlizada. Para ello se deberán generar distintos gráficos que representen las respuestas a las siguientes preguntas guía:

1. Acerca de pokebolas:
- a) Ejecutando la función 100 veces, para cada Pokemon en condiciones ideales (HP:100 %, LVL 100) ¿Cuál es la probabilidad de captura promedio para cada pokebola?

In [ ]:
import json

factory = PokemonFactory("pokemon.json")
with open("pokemon.json", "r") as c:
            pokemon_db = json.load(c) 
pokemons = pokemon_db.keys()

pokemons

In [ ]:
pokeball_catch_rate = pd.DataFrame(columns=[])
for pokemon_name in pokemons:
    pokemon = factory.create(pokemon_name, 100, StatusEffect.NONE, 1)
    for _ in range(100):
        catchT1, valor1 = attempt_catch(pokemon, "pokeball")
        catchT2, valor2 = attempt_catch(pokemon, "ultraball")
        catchT3, valor3 = attempt_catch(pokemon, "fastball")
        catchT4, valor4 = attempt_catch(pokemon, "heavyball")
        new_row = {"pokemon_name": pokemon_name, "hp_percentage":1, "pokeball": valor1, "Capturado_pb":catchT1, "ultraball": valor2, "Capturado_ub":catchT2, "fastball": valor3, "Capturado_fb":catchT3, "heavyball": valor4,"Capturado_hb":catchT4  }
        pokeball_catch_rate = pd.concat([pokeball_catch_rate, pd.DataFrame([new_row])], ignore_index=True)
        
columnas_numericas = pokeball_catch_rate.select_dtypes(include=['float64'])     
promedios = columnas_numericas.mean()
promedios
#pokeball_catch_rate     #Tiene fijo status y hp

In [ ]:
import matplotlib.pyplot as plt
promedios_list = promedios.to_list()
plt.bar(promedios.index, promedios_list)
plt.xlabel('Pokeballs')
plt.ylabel('Promedio')
plt.title('Promedios por Pokeball')
plt.show()

Lo que se observa es que lo general las ultraball tiene mayor promedio de probabilidad de captura.

- b) ¿Es cierto que algunas pokebolas son más o menos efectivas dependiendo de propiedades intrínsecas de cada Pokemon? Justificar.
Sugerencia: Comparar efectividad (success/total attemps) como proporción de la efectividad de la Pokebola básica para cada Pokemon

In [ ]:
# Filtrar las columnas relevantes
columnas_capturado = pokeball_catch_rate[['pokemon_name', 'Capturado_pb', 'Capturado_ub', 'Capturado_fb', 'Capturado_hb']]

# Contar los valores True agrupados por nombre de Pokémon
conteo_por_pokemon = ( columnas_capturado.groupby('pokemon_name').sum() )/100

conteo_por_pokemon

In [ ]:
df = pd.DataFrame(conteo_por_pokemon)
# Cambiar nombres de las columnas
nuevos_nombres = {'Capturado_pb': 'Pokeball', 'Capturado_ub': 'Ultraball', 'Capturado_fb': 'Fastball','Capturado_hb': 'Heavyball'}
df = df.rename(columns=nuevos_nombres)

# Crear figura y ejes 3D
fig = plt.figure(figsize=(6, 8))
ax = fig.add_subplot(111, projection='3d')

# Configurar etiquetas de los ejes
x_labels = df.index.tolist()
y_labels = df.columns.tolist()

# Agregar líneas para cada serie de datos
for i, col in enumerate(df.columns):
    ax.plot(range(len(df)), [i] * len(df), df[col], label=col)

# Configurar etiquetas de los ejes
ax.set_xticks(range(len(df)))
ax.set_xticklabels(x_labels, rotation=45)
ax.set_yticks(range(len(df.columns)))
ax.set_yticklabels(y_labels)

# Configurar etiquetas y título
#ax.set_xlabel('Índices')
#ax.set_ylabel('Columnas')
ax.set_zlabel('Efectividad', labelpad=1)
ax.set_title('Gráfico Efectividad')

# Agregar leyenda
ax.legend()

# Mostrar el gráfico
#plt.tight_layout()  # Ajustar el diseño para evitar superposición de elementos
plt.show()



Lo que se observa es que la efectividad de cada tipo de pokebola si depende de cual pokemos queremos capturar, dado que las características intrinsecas de cada pokemos es relevante. No es lo mismo querer capturar un caterpie en diferencia a un mewtwo.

2. Acerca del estado del Pokemon:
- a) ¿Las condiciones de salud tienen algún efecto sobre la efectividad de la captura? Si es así, ¿Cúal es más o menos efectiva?

In [ ]:
# Vario el hp_percentage
pokeball_catch_rate2 = pd.DataFrame(columns=[])
for pokemon_name in pokemons:
    for hp in np.arange(0, 1, 0.1):
        pokemon = factory.create(pokemon_name, 100, StatusEffect.NONE, hp)
        for _ in range(100):
            catchT1, valor1 = attempt_catch(pokemon, "pokeball")
            catchT2, valor2 = attempt_catch(pokemon, "ultraball")
            catchT3, valor3 = attempt_catch(pokemon, "fastball")
            catchT4, valor4 =attempt_catch(pokemon, "heavyball")
            new_row = {"pokemon_name": pokemon_name, "hp_percentage":hp,"pokeball": valor1, "Capturado_pb":catchT1, "ultraball": valor2, "Capturado_ub":catchT2, "fastball": valor3, "Capturado_fb":catchT3, "heavyball": valor4,"Capturado_hb":catchT4  }
            pokeball_catch_rate2 = pd.concat([pokeball_catch_rate2, pd.DataFrame([new_row])], ignore_index=True)
        
#columnas_numericas2 = pokeball_catch_rate2.select_dtypes(include=['float64'])     
#promedios2 = columnas_numericas2.mean()
#
#pokeball_catch_rate2

In [ ]:
# Filtrar las filas con valores True en las columnas de interés
capturado_columns = ['pokemon_name','hp_percentage','Capturado_pb', 'Capturado_ub', 'Capturado_fb', 'Capturado_hb']
df_capturado = pokeball_catch_rate2[capturado_columns]

# Contar la cantidad de True por cada pokemon_name y hp_percentage
result = df_capturado.groupby(['pokemon_name', 'hp_percentage']).sum() / 100
result

In [ ]:
#para hace la grafica, restablezco los indices a columnas
result1 = result.reset_index()
result1

In [ ]:
df = pd.DataFrame(result1)
df.rename(columns={'Capturado_pb':'Pokeball',	'Capturado_ub':'Ultraball',	'Capturado_fb':'Fastball',	'Capturado_hb':'Heavyball'}, inplace=True)
#['Pokeball',	'Ultraball',	'Fastball',	'Heavyball']
# Crear una matriz de 2x2 con subgráficos
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

for i, ball_type in enumerate(['Pokeball',	'Ultraball',	'Fastball',	'Heavyball']):
    row, col = divmod(i, 2)
    for pokemon in df['pokemon_name'].unique():
        subset = df[df['pokemon_name'] == pokemon]
        axs[row, col].plot(subset['hp_percentage'], subset[ball_type], label=pokemon)
    axs[row, col].set_xlabel('HP %')
    axs[row, col].set_ylabel('Efectividad de la Bola')
    axs[row, col].set_title(f'{ball_type.capitalize()}')
    axs[row, col].legend()

plt.tight_layout()
plt.show()

Lo que se observa es que en en general a medidad que aumenta el Hp % disminuye la efectividad de la bola. Aunque en el caso de la ultrabola sigue siendo medianamente efectiva.

- b) ¿Cómo afectan los puntos de vida a la efectividad de la captura?
Sugerencia: Elegir uno o dos Pokemones y manteniendo el resto de los parámetros constantes, calcular la probabilidad de captura para distintos HP %

- -  caterpie

In [ ]:
#vario status y hp_percentage
pokeball_catch_rate4 = pd.DataFrame([])
status = StatusEffect

pokemon_name="caterpie"
for n in np.arange(0,1,0.2):                                            #6
    for hp in np.arange(0, 1, 0.1):                         #10
        pokemon = factory.create(pokemon_name, 100, StatusEffect.NONE, hp)       
        for _ in range(100):
            catchT1, valor1 = attempt_catch(pokemon, "pokeball",n)
            catchT2, valor2 = attempt_catch(pokemon, "ultraball",n)
            catchT3, valor3 = attempt_catch(pokemon, "fastball",n)
            catchT4, valor4 =attempt_catch(pokemon, "heavyball",n)
            new_row = {"pokemon_name": pokemon_name, "noise":n, "hp_percentage":hp, "pokeball": valor1, "Capturado_pb":catchT1, "ultraball": valor2, "Capturado_ub":catchT2, "fastball": valor3, "Capturado_fb":catchT3, "heavyball": valor4,"Capturado_hb":catchT4  }
            pokeball_catch_rate4 = pd.concat([pokeball_catch_rate4, pd.DataFrame([new_row])], ignore_index=True)

# Filtrar las filas con valores True en las columnas de interés
capturado_columns = ['noise', 'hp_percentage','Capturado_pb', 'Capturado_ub', 'Capturado_fb', 'Capturado_hb']
df_capturado = pokeball_catch_rate4[capturado_columns]

# Contar la cantidad de True por cada pokemon_name y hp_percentage
result3 = df_capturado.groupby(['noise', 'hp_percentage']).sum() / 100
#result3
pokeball_catch_rate4


In [ ]:
#para hace la grafica, restablezco los indices a columnas
result3g = result3.reset_index()
result3g

In [ ]:
df = pd.DataFrame(result3g)
df.rename(columns={'Capturado_pb':'Pokeball',	'Capturado_ub':'Ultraball',	'Capturado_fb':'Fastball',	'Capturado_hb':'Heavyball'}, inplace=True)
#['Pokeball',	'Ultraball',	'Fastball',	'Heavyball']
# Crear una matriz de 2x2 con subgráficos
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

for i, ball_type in enumerate(['Pokeball',	'Ultraball',	'Fastball',	'Heavyball']):
    row, col = divmod(i, 2)
    for n in df['noise'].unique():
        subset = df[df['noise'] == n]
        axs[row, col].plot(subset['hp_percentage'], subset[ball_type], label=n)
    axs[row, col].set_xlabel('HP %')
    axs[row, col].set_ylabel('Efectividad de la Bola')
    axs[row, col].set_title(f'{ball_type.capitalize()}')
    axs[row, col].legend()

plt.tight_layout()
plt.show()

- - jolteon

In [ ]:
#vario status y hp_percentage
pokeball_catch_rate5 = pd.DataFrame([])
status = StatusEffect

pokemon_name="jolteon"
for n in np.arange(0,1,0.2):                                            #6
    for hp in np.arange(0, 1, 0.1):                         #10
        pokemon = factory.create(pokemon_name, 100, StatusEffect.NONE, hp)       
        for _ in range(100):
            catchT1, valor1 = attempt_catch(pokemon, "pokeball",n)
            catchT2, valor2 = attempt_catch(pokemon, "ultraball",n)
            catchT3, valor3 = attempt_catch(pokemon, "fastball",n)
            catchT4, valor4 =attempt_catch(pokemon, "heavyball",n)
            new_row = {"pokemon_name": pokemon_name, "noise":n, "hp_percentage":hp, "pokeball": valor1, "Capturado_pb":catchT1, "ultraball": valor2, "Capturado_ub":catchT2, "fastball": valor3, "Capturado_fb":catchT3, "heavyball": valor4,"Capturado_hb":catchT4  }
            pokeball_catch_rate5 = pd.concat([pokeball_catch_rate5, pd.DataFrame([new_row])], ignore_index=True)

# Filtrar las filas con valores True en las columnas de interés
capturado_columns = ['noise', 'hp_percentage','Capturado_pb', 'Capturado_ub', 'Capturado_fb', 'Capturado_hb']
df_capturado = pokeball_catch_rate5[capturado_columns]

# Contar la cantidad de True por cada pokemon_name y hp_percentage
result4 = df_capturado.groupby(['noise', 'hp_percentage']).sum() / 100
#result3
pokeball_catch_rate5


In [ ]:
#para hace la grafica, restablezco los indices a columnas
result4g = result4.reset_index()
result4g

In [ ]:
df = pd.DataFrame(result4g)
df.rename(columns={'Capturado_pb':'Pokeball',	'Capturado_ub':'Ultraball',	'Capturado_fb':'Fastball',	'Capturado_hb':'Heavyball'}, inplace=True)
#['Pokeball',	'Ultraball',	'Fastball',	'Heavyball']
# Crear una matriz de 2x2 con subgráficos
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

for i, ball_type in enumerate(['Pokeball',	'Ultraball',	'Fastball',	'Heavyball']):
    row, col = divmod(i, 2)
    for n in df['noise'].unique():
        subset = df[df['noise'] == n]
        axs[row, col].plot(subset['hp_percentage'], subset[ball_type], label=n)
    axs[row, col].set_xlabel('HP %')
    axs[row, col].set_ylabel('Efectividad de la Bola')
    axs[row, col].set_title(f'{ball_type.capitalize()}')
    axs[row, col].legend()

plt.tight_layout()
plt.show()

Seteando el valor de ruido se observa que: BLA BLA BLA BLA

- c) ¿Qué parámetros son los que más afectan la probabilidad de captura?

In [ ]:
#vario status y hp_percentage
pokeball_catch_rate6 = pd.DataFrame([])
status = StatusEffect

for pokemon_name in pokemons:
    for n in np.arange(0,1,0.1):                                             #6
        for hp in np.arange(0, 1, 0.1):                         #10
            pokemon = factory.create(pokemon_name, 100, StatusEffect.NONE, hp)       
            for _ in range(1):
                catchT1, valor1 = attempt_catch(pokemon, "pokeball",n)
                catchT2, valor2 = attempt_catch(pokemon, "ultraball",n)
                catchT3, valor3 = attempt_catch(pokemon, "fastball",n)
                catchT4, valor4 =attempt_catch(pokemon, "heavyball",n)
                new_row = {"pokemon_name": pokemon_name, "noise":n, "hp_percentage":hp, "pokeball": valor1, "Capturado_pb":catchT1, "ultraball": valor2, "Capturado_ub":catchT2, "fastball": valor3, "Capturado_fb":catchT3, "heavyball": valor4,"Capturado_hb":catchT4  }
                pokeball_catch_rate6 = pd.concat([pokeball_catch_rate6, pd.DataFrame([new_row])], ignore_index=True)

# Filtrar las filas con valores True en las columnas de interés
#capturado_columns = ['status', 'hp_percentage','Capturado_pb', 'Capturado_ub', 'Capturado_fb', 'Capturado_hb']
#df_capturado = pokeball_catch_rate6[capturado_columns]

# Contar la cantidad de True por cada pokemon_name y hp_percentage
#result4 = df_capturado.groupby(['status', 'hp_percentage']).sum() / 100
#result4
pokeball_catch_rate6

In [ ]:
df_pokeball = pokeball_catch_rate6[['pokemon_name','noise','hp_percentage','pokeball']]
df_pokeball

In [ ]:
dfg = pd.DataFrame(df_pokeball)

j=0
for p in pokemons:
    dfg['pokemon_name'] = dfg['pokemon_name'].replace(p, j)
    j=j+1
dfg

In [ ]:
# Crea una figura 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Grafica los puntos
scatter = ax.scatter(dfg['pokemon_name'], dfg['noise'], dfg['pokeball'], c=dfg['hp_percentage'], cmap='viridis')

# Configura los ejes
#ax.set_xlabel('Pokemon name')
ax.set_ylabel('Ruido')
ax.set_zlabel('Pokeball Value',labelpad=1)

ax.set_title('Gráfico de Prop.de Capt. de Pokeball')

# Configurar etiquetas de los ejes
x_labels = pokemons
#y_labels = dfg['noise'].unique()
y_labels = ['0.0','0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9']

# Agrega una leyenda para el color
legend = ax.legend(*scatter.legend_elements(), title='HP %')
ax.add_artist(legend)
# Configurar etiquetas de los ejes
ax.set_xticks(range(5))
ax.set_xticklabels(x_labels, rotation=45)
#ax.set_yticks(range(10))
#ax.set_yticklabels(y_labels)


# Muestra el gráfico
plt.show()

En el grafico de arriba comparo los diferentes parametros (de nuevo varie el status y no el noise)
Se observa lo siguiente:
BLA BLA BLA.

A continuación realizao un analisis para saber como varia cada variable en relación a la otra.
Esto es: analisis de covarianza y correlación. Enfocandome en la correlación para saber como la Probabilidad de Captura varia en relación a las demas parametros mutables.

In [ ]:
dfg.rename(columns={'pokemon_name': 'Pokemon','hp_percentage':'HP %', 'pokeball':'Prop.Capt.'}, inplace=True)

In [ ]:
covariance_matrix = dfg.cov()
covariance_matrix

In [ ]:
sns.heatmap(covariance_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Covarianza de una Pokeball')
plt.show()

In [ ]:
correlation_matrix = dfg.corr()
correlation_matrix

In [ ]:
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación de una Pokeball')
plt.show()

Leyendo las celdas de la fila inferior correspondiente a Prop.Capt, se observa que el mayor valor de correlación es "status".
Esto es para PokeBall

Leyendo las celdas de la fila inferior correspondiente a Prop.Capt, se observa que el mayor valor de correlación es "status".
Esto es para UltraBall

In [ ]:
#PARA FASTBALL
df_fastball = pokeball_catch_rate6[['pokemon_name','noise','hp_percentage','fastball']]

dfg = pd.DataFrame(df_fastball)


i=0
for p in pokemons:
    dfg['pokemon_name'] = dfg['pokemon_name'].replace(p, i)
    i=i+1

dfg.rename(columns={'pokemon_name': 'Pokemon','hp_percentage':'HP %', 'fastball':'Prop.Capt.'}, inplace=True)

correlation_matrix = dfg.corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación de una Fastball')
plt.show()

Leyendo las celdas de la fila inferior correspondiente a Prop.Capt, se observa que el mayor valor de correlación es "status".
Esto es para FastBall

In [ ]:
#PARA HEAVYBALL
df_heavyball = pokeball_catch_rate6[['pokemon_name','noise','hp_percentage','heavyball']]

dfg = pd.DataFrame(df_heavyball)

i=0
for p in pokemons:
    dfg['pokemon_name'] = dfg['pokemon_name'].replace(p, i)
    i=i+1

dfg.rename(columns={'pokemon_name': 'Pokemon','hp_percentage':'HP %', 'heavyball':'Prop.Capt.'}, inplace=True)

correlation_matrix = dfg.corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación de una Heavyball')
plt.show()

Leyendo las celdas de la fila inferior correspondiente a Prop.Capt, se observa que el mayor valor de correlación es "status".
Esto es para HeavyBall.

- d) Teniendo en cuenta uno o dos pokemones distintos: ¿Qué combinación de condiciones (propiedades mutables) y pokebola conviene utilizar para capturarlos?

Recordando cual parametros son mutables y cuales no:
-                no mutable     name: str,
-                no mutable     type: Tuple[Type, Type],        # Especie
*                mutable        current_hp : int,               # Vida actual    *
*                mutable        status_effect : StatusEffect,   # Condición de salud
*                mutable        level : int,                    # Nivel de experiencia
-                no mutable     stats: Stats,                   # [base hp, attack, defense, Sp. attack, Sp. defense, speed]
-                no mutable     catch_rate: int,                # Dificultad de captura. (Min: 0, Max: 259
-                no mutable     weight: float,                  # Peso

Lo que vi leyendo el codigo, es que: el level no afecta a la probibildad / efectividad de captura. ¿ O estoy equivocado ?

In [ ]:
conteo_por_pokemon_t = conteo_por_pokemon.reset_index()
conteo_por_pokemon_t


- e) A partir del punto anterior, ¿sería efectiva otra combinación de parámetros teniendo en cuenta un nivel del pokemon más bajo (o más alto)?